# Settings

## Constants

In [26]:
import os

PATH_DATA = './Dataset'
sensor_data = ['ParticipantInfo.csv', 'Service.csv', 'ContextualFactor.csv', 'Availability.csv']

# Utility Functions

In [27]:
# Install required packages
!pip install -q scikit-learn catboost
!pip install -q scikit-learn catboost xgboost lightgbm imbalanced-learn

# === Core Libraries ===
import pandas as pd
import numpy as np
from datetime import datetime, timezone, timedelta

# Set timezone to UTC+9 for consistent time handling
tz = timezone(+timedelta(hours=9))

# === Preprocessing and Utilities ===
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict

# === Model Selection and Evaluation ===
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.metrics import accuracy_score, f1_score

# === Machine Learning Models ===
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.svm import SVC
from sklearn.dummy import DummyClassifier

# === Handling Imbalanced Data ===
from imblearn.over_sampling import SMOTE


# Load the Dataset into Dataframe

In [28]:
dataframes = {
    filename: pd.read_csv(os.path.join(PATH_DATA, filename)).reset_index(drop=True)
    for filename in sensor_data
}
dfService = dataframes['Service.csv']
dfContextualFactor = dataframes['ContextualFactor.csv']
dfParticipantInfo = dataframes['ParticipantInfo.csv']
dfAvailability = dataframes['Availability.csv']


# Preprocessing

In [29]:
# Select specific columns from dfContextualFactor, dfService, and dfUserInfo DataFrames
# Combine relevant columns to create a unified dataset
dfContextualFactor_selected_columns=dfContextualFactor[['pid','sid','activity1','activity2','activity3','participantRoom','participantPosition',]]
dfService_selected_columns=dfService[['weekOfExperiment','dayOfWeek','startTime', 'activityInquiry','availabilityInquiry','speechShadowing','continue-to-nextInquiry_1','continue-to-nextInquiry_2','continue-to-nextInquiry_3','continue-to-nextInquiry_4','endTime','endType']]

dfCombinedAll=pd.concat([dfContextualFactor_selected_columns, dfService_selected_columns], axis=1)

In [30]:
# Concatenate and merge the all columns from dfParticipantInfo(ParticipantInfo.csv)
dfParticipantInfo_selected_columns = dfParticipantInfo[['pid', 'homeType', 'speakerRoom', 'speakerPosition']]
dfCombinedAll = pd.merge(dfCombinedAll, dfParticipantInfo_selected_columns, on='pid', how='left')

## Position Processing

In [31]:
# Define function to calculate proximity between user and speaker
def calculate_proximity(row):
    # Return 0 if user and speaker are in different rooms
    if row['participantRoom'] != row['speakerRoom']:
        return 0
    # Return 1 if in same room but userPosition is missing
    elif pd.isna(row['participantPosition']) or row['participantPosition'].strip() == '':
        return 1
    # Return 3 if in same room and same position
    elif row['participantPosition'] == row['speakerPosition']:
        return 3
    # Return 2 if in same room but different positions
    else:
        return 2

# Apply proximity calculation to create a new 'proximity' column
dfCombinedAll['proximity'] = dfCombinedAll.apply(calculate_proximity, axis=1)
print(dfCombinedAll[['participantRoom', 'participantPosition', 'speakerRoom', 'speakerPosition', 'proximity']].head(10))


  participantRoom participantPosition speakerRoom speakerPosition  proximity
0        Bed Room                 Bed    Bed Room            Desk          2
1       Rest Room                 NaN    Bed Room            Desk          0
2     Living Room                 NaN    Bed Room            Desk          0
3     Living Room                 NaN    Bed Room            Desk          0
4        Bed Room                 Bed    Bed Room            Desk          2
5        Bed Room                 Bed    Bed Room            Desk          2
6        Bed Room                 Bed    Bed Room            Desk          2
7        Bed Room                Desk    Bed Room            Desk          3
8        Bed Room                 Bed    Bed Room            Desk          2
9        Bed Room                 Bed    Bed Room            Desk          2


## Activity and Time Processing

In [32]:
# Process activity columns for one-hot encoding
activity_cols = ['activity1', 'activity2', 'activity3']
df_activity = dfContextualFactor[activity_cols].copy()

# Get unique activities across all activity columns, excluding NaN
all_unique_activities = pd.unique(df_activity.values.ravel())
all_unique_activities = [x for x in all_unique_activities if pd.notna(x)]

# Create a DataFrame for one-hot encoding of activities
dfActivity_one_hot_encoding = pd.DataFrame(0, index=df_activity.index, columns=['act_' + str(val) for val in all_unique_activities])

# Perform one-hot encoding for each activity column
for col in activity_cols:
    for val in all_unique_activities:
        dfActivity_one_hot_encoding['act_' + str(val)] |= (df_activity[col] == val).astype(int)

# Concatenate one-hot encoded activity columns to dfCombinedAll
dfActivity_one_hot_encoding
dfCombinedAll = pd.concat([dfCombinedAll, dfActivity_one_hot_encoding], axis=1)

In [33]:
# Convert startTime to datetime and extract total minutes since midnight
dfCombinedAll['startTime'] = pd.to_datetime(dfCombinedAll['startTime'], format='%H:%M:%S', errors='coerce')
dfCombinedAll['minute'] = dfCombinedAll['startTime'].dt.hour * 60 + dfCombinedAll['startTime'].dt.minute

# Map days of the week to numerical values (MON=0, TUE=1, ..., SUN=6)
day_map = {'MON': 0, 'TUE': 1, 'WED': 2, 'THU': 3, 'FRI': 4, 'SAT': 5, 'SUN': 6}
dfCombinedAll['dayOfWeek'] = dfCombinedAll['dayOfWeek'].map(day_map)

# Bin minutes into 30-minute intervals for temporal analysis
dfCombinedAll['minute_bin'] = (dfCombinedAll['minute'] // 30).astype(int)
dfCombinedAll.head()

,pid,sid,activity1,activity2,activity3,participantRoom,participantPosition,weekOfExperiment,dayOfWeek,startTime,...,act_Social Interaction,act_Returning from Outside / other Room,act_Studying / Working,act_Others,act_House Chores,act_Self Caring,act_Visiting Outside / Other Room,act_Resting,minute,minute_bin
0,1,1,Taking a Nap / Sleeping,NaN,NaN,Bed Room,Bed,1,0,1900-01-01 09:00:26,...,0,0,0,0,0,0,0,0,540,18
1,1,2,Hygiene,NaN,NaN,Rest Room,NaN,1,0,1900-01-01 10:34:26,...,0,0,0,0,0,0,0,0,634,21
2,1,3,Eating,NaN,NaN,Living Room,NaN,1,1,1900-01-01 09:00:46,...,0,0,0,0,0,0,0,0,540,18
3,1,4,Eating,NaN,NaN,Living Room,NaN,1,1,1900-01-01 09:44:46,...,0,0,0,0,0,0,0,0,584,19
4,1,5,Taking a Nap / Sleeping,NaN,NaN,Bed Room,Bed,1,1,1900-01-01 12:04:48,...,0,0,0,0,0,0,0,0,724,24


## SHORT_INTERACTION and LONT_INTERACTION

In [34]:
# Create binary columns for interaction types
# SHORT_INTERACTION: True if availabilityInquiry is not NaN
dfCombinedAll['SHORT_INTERACTION_availability'] = dfCombinedAll['availabilityInquiry'].notna()

# LONG_INTERACTION: True if continue-to-nextInquiry_1 is not NaN
dfCombinedAll['LONG_INTERACTION_availability'] = dfCombinedAll['continue-to-nextInquiry_1'].notna()

In [35]:
dfCombinedAll.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2905 entries, 0 to 2904
Data columns (total 39 columns):
 #   Column                                   Non-Null Count  Dtype         
---  ------                                   --------------  -----         
 0   pid                                      2905 non-null   int64         
 1   sid                                      2905 non-null   int64         
 2   activity1                                2905 non-null   object        
 3   activity2                                112 non-null    object        
 4   activity3                                3 non-null      object        
 5   participantRoom                          2905 non-null   object        
 6   participantPosition                      2414 non-null   object        
 7   weekOfExperiment                         2905 non-null   int64         
 8   dayOfWeek                                2905 non-null   int64         
 9   startTime                                

# Feature Extraction

In [36]:
# Select features for response prediction
dfFeatresForResponse = dfCombinedAll[['pid',
    'act_Taking a Nap / Sleeping','act_Hygiene','act_Eating','act_Using Media','act_Social Interaction',
    'act_Returning from Outside / other Room','act_Studying / Working','act_Others','act_House Chores',
    'act_Self Caring','act_Visiting Outside / Other Room','act_Resting',
    'homeType','participantRoom','participantPosition','speakerRoom','speakerPosition',
    'minute_bin','dayOfWeek','SHORT_INTERACTION_availability','LONG_INTERACTION_availability']].copy()

# Encode categorical columns using LabelEncoder
categorical_columns = ['homeType', 'participantRoom', 'participantPosition', 'speakerRoom', 'speakerPosition', 'minute_bin']
label_encoders = defaultdict(LabelEncoder)

# Apply label encoding to each categorical column
for col in categorical_columns:
    dfFeatresForResponse[col] = label_encoders[col].fit_transform(dfFeatresForResponse[col])

# Create a copy of the encoded data for further processing
encoded_data = dfFeatresForResponse.copy()


encoded_data.info()
encoded_data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2905 entries, 0 to 2904
Data columns (total 22 columns):
 #   Column                                   Non-Null Count  Dtype
---  ------                                   --------------  -----
 0   pid                                      2905 non-null   int64
 1   act_Taking a Nap / Sleeping              2905 non-null   int64
 2   act_Hygiene                              2905 non-null   int64
 3   act_Eating                               2905 non-null   int64
 4   act_Using Media                          2905 non-null   int64
 5   act_Social Interaction                   2905 non-null   int64
 6   act_Returning from Outside / other Room  2905 non-null   int64
 7   act_Studying / Working                   2905 non-null   int64
 8   act_Others                               2905 non-null   int64
 9   act_House Chores                         2905 non-null   int64
 10  act_Self Caring                          2905 non-null   int64
 11  act_

,pid,act_Taking a Nap / Sleeping,act_Hygiene,act_Eating,act_Using Media,act_Social Interaction,act_Returning from Outside / other Room,act_Studying / Working,act_Others,act_House Chores,...,act_Resting,homeType,participantRoom,participantPosition,speakerRoom,speakerPosition,minute_bin,dayOfWeek,SHORT_INTERACTION_availability,LONG_INTERACTION_availability
0,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,12,0,True,False
1,1,0,1,0,0,0,0,0,0,0,...,0,0,3,3,0,1,15,0,False,False
2,1,0,0,1,0,0,0,0,0,0,...,0,0,1,3,0,1,12,1,False,False
3,1,0,0,1,0,0,0,0,0,0,...,0,0,1,3,0,1,13,1,False,False
4,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,18,1,True,False


## Label: SHORT_INTERACTION

In [37]:
print(encoded_data['SHORT_INTERACTION_availability'].value_counts())

SHORT_INTERACTION_availability
True     2159
False     746
Name: count, dtype: int64


### Model Building and LOSO CV

In [38]:
# Prepare features (X) and target variables (y) for SHORT_INTERACTION
X = encoded_data.drop(columns=['SHORT_INTERACTION_availability', 'LONG_INTERACTION_availability', 'pid'], axis=1) # Drop target and unrelated columns
y = encoded_data['SHORT_INTERACTION_availability'] # Target variable
groups = encoded_data['pid'] # Group by user ID for Leave-One-Group-Out CV

# Initialize Leave-One-Group-Out cross-validator
logo = LeaveOneGroupOut()

# Initialize SMOTE for oversampling to handle class imbalance
smote = SMOTE(random_state=42)

# Define models to evaluate
models1 = {
    'Random Forest': RandomForestClassifier(n_estimators=100, max_depth=5, random_state=42),
    'Gradient Boosting': GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42),
    'XGBoost': XGBClassifier(n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42),
    'LightGBM': LGBMClassifier(n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42),
    'CatBoost': CatBoostClassifier(iterations=100, depth=5, learning_rate=0.1, loss_function='Logloss', cat_features=[0], random_seed=42),
    'SVM': SVC(random_state=42),
    'Dummy': DummyClassifier(strategy="stratified", random_state=42)
}

results1 = {}

# Loop over each model
for model_name, model1 in models1.items():
    accuracies = []
    f1_scores = []

    # Loop over each fold in Leave-One-Group-Out cross-validation
    for i, (train_index, test_index) in enumerate(logo.split(X, y, groups)):
        # Split the data into training and testing sets for the current fold
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        # Apply SMOTE to the training data to balance class distribution
        X_train_oversampled, y_train_oversampled = smote.fit_resample(X_train, y_train)

        # Train the model on the oversampled training data
        model1.fit(X_train_oversampled, y_train_oversampled.ravel())

        # Predict the target on the test data
        y_pred = model1.predict(X_test)

        # Evaluate the prediction
        accuracy = accuracy_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred, average='macro')

        accuracies.append(accuracy)
        f1_scores.append(f1)
    # Compute average accuracy and F1-score across all folds
    avg_accuracy = sum(accuracies) / len(accuracies)
    avg_f1_score = sum(f1_scores) / len(f1_scores)

    results1[model_name] = {
        'Average Accuracy': avg_accuracy,
        'Average F1-Score (macro)': avg_f1_score
    }

for model_name, metrics in results1.items():
    print(f'{model_name} - Accuracy: {metrics["Average Accuracy"]:.4f}, F1-Score: {metrics["Average F1-Score (macro)"]:.4f}')

C:\Users\Golibjon\AppData\Local\Temp\ipykernel_18052\3916840703.py:40: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model1.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_18052\3916840703.py:40: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model1.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_18052\3916840703.py:40: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model1.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_18052\3916840703.py:40: FutureWarning: Series.ravel is d

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 2105, number of negative: 2105
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000557 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 90
[LightGBM] [Info] Number of data points in the train set: 4210, number of used features: 18
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with pos

C:\Users\Golibjon\AppData\Local\Temp\ipykernel_18052\3916840703.py:40: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model1.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_18052\3916840703.py:40: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model1.fit(X_train_oversampled, y_train_oversampled.ravel())


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

C:\Users\Golibjon\AppData\Local\Temp\ipykernel_18052\3916840703.py:40: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model1.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_18052\3916840703.py:40: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model1.fit(X_train_oversampled, y_train_oversampled.ravel())


...

C:\Users\Golibjon\AppData\Local\Temp\ipykernel_18052\3916840703.py:40: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model1.fit(X_train_oversampled, y_train_oversampled.ravel())


0:	learn: 0.6559277	total: 1.34ms	remaining: 133ms
1:	learn: 0.6229568	total: 2.57ms	remaining: 126ms
2:	learn: 0.5929909	total: 3.78ms	remaining: 122ms
3:	learn: 0.5720313	total: 4.96ms	remaining: 119ms
4:	learn: 0.5524066	total: 6.33ms	remaining: 120ms
5:	learn: 0.5341658	total: 7.45ms	remaining: 117ms
6:	learn: 0.5141029	total: 8.55ms	remaining: 114ms
7:	learn: 0.4987436	total: 9.95ms	remaining: 114ms
8:	learn: 0.4843780	total: 11.1ms	remaining: 112ms
9:	learn: 0.4769646	total: 11.9ms	remaining: 107ms
10:	learn: 0.4658189	total: 13.2ms	remaining: 107ms
11:	learn: 0.4571497	total: 14.5ms	remaining: 106ms
12:	learn: 0.4478482	total: 15.8ms	remaining: 106ms
13:	learn: 0.4421657	total: 17ms	remaining: 105ms
14:	learn: 0.4359737	total: 18.4ms	remaining: 105ms
15:	learn: 0.4291168	total: 19.7ms	remaining: 103ms
16:	learn: 0.4230028	total: 21.2ms	remaining: 103ms
17:	learn: 0.4182240	total: 22.4ms	remaining: 102ms
18:	learn: 0.4135874	total: 23.6ms	remaining: 101ms
19:	learn: 0.4103966	tot

C:\Users\Golibjon\AppData\Local\Temp\ipykernel_18052\3916840703.py:40: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model1.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_18052\3916840703.py:40: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model1.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_18052\3916840703.py:40: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model1.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_18052\3916840703.py:40: FutureWarning: Series.ravel is d

Random Forest - Accuracy: 0.8326, F1-Score: 0.7380
Gradient Boosting - Accuracy: 0.8163, F1-Score: 0.7122
XGBoost - Accuracy: 0.8182, F1-Score: 0.7130
LightGBM - Accuracy: 0.8147, F1-Score: 0.7120
CatBoost - Accuracy: 0.8125, F1-Score: 0.7156
SVM - Accuracy: 0.7517, F1-Score: 0.6726
Dummy - Accuracy: 0.5064, F1-Score: 0.4413


C:\Users\Golibjon\AppData\Local\Temp\ipykernel_18052\3916840703.py:40: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model1.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_18052\3916840703.py:40: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model1.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_18052\3916840703.py:40: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model1.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_18052\3916840703.py:40: FutureWarning: Series.ravel is d

### K-Fold CV

In [39]:
from sklearn.model_selection import KFold

# Initialize 5-fold cross-validator
kfold = KFold(n_splits=5, shuffle=False)

results1 = {}

# Loop over each model
for model_name, model1 in models1.items():
    accuracies = []
    f1_scores = []

    # Loop over each fold in 5-fold cross-validation
    for train_index, test_index in kfold.split(X, y):
        # Split the data into training and testing sets for the current fold
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        # Apply SMOTE to the training data to balance class distribution
        X_train_oversampled, y_train_oversampled = smote.fit_resample(X_train, y_train)

        # Train the model on the oversampled training data
        model1.fit(X_train_oversampled, y_train_oversampled.ravel())

        # Predict the target on the test data
        y_pred = model1.predict(X_test)

        # Evaluate the prediction
        accuracy = accuracy_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred, average='macro')

        accuracies.append(accuracy)
        f1_scores.append(f1)

    # Compute average accuracy and F1-score across all folds
    avg_accuracy = sum(accuracies) / len(accuracies)
    avg_f1_score = sum(f1_scores) / len(f1_scores)

    results1[model_name] = {
        'Average Accuracy': avg_accuracy,
        'Average F1-Score (macro)': avg_f1_score
    }

print()
for model_name, metrics in results1.items():
    print(f'{model_name} - Accuracy: {metrics["Average Accuracy"]:.4f}, F1-Score: {metrics["Average F1-Score (macro)"]:.4f}')

C:\Users\Golibjon\AppData\Local\Temp\ipykernel_18052\1622768231.py:23: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model1.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_18052\1622768231.py:23: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model1.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_18052\1622768231.py:23: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model1.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_18052\1622768231.py:23: FutureWarning: Series.ravel is d

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 1717, number of negative: 1717
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000550 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 88
[LightGBM] [Info] Number of data points in the train set: 3434, number of used features: 18
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with pos

C:\Users\Golibjon\AppData\Local\Temp\ipykernel_18052\1622768231.py:23: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model1.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_18052\1622768231.py:23: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model1.fit(X_train_oversampled, y_train_oversampled.ravel())


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

C:\Users\Golibjon\AppData\Local\Temp\ipykernel_18052\1622768231.py:23: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model1.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_18052\1622768231.py:23: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model1.fit(X_train_oversampled, y_train_oversampled.ravel())


...

C:\Users\Golibjon\AppData\Local\Temp\ipykernel_18052\1622768231.py:23: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model1.fit(X_train_oversampled, y_train_oversampled.ravel())


16:	learn: 0.4180250	total: 20ms	remaining: 97.6ms
17:	learn: 0.4131113	total: 21.3ms	remaining: 97ms
18:	learn: 0.4098017	total: 22.5ms	remaining: 95.7ms
19:	learn: 0.4052812	total: 23.6ms	remaining: 94.4ms
20:	learn: 0.4025855	total: 24.7ms	remaining: 92.9ms
21:	learn: 0.4002905	total: 25.9ms	remaining: 91.7ms
22:	learn: 0.3983489	total: 26.9ms	remaining: 90.2ms
23:	learn: 0.3951260	total: 28ms	remaining: 88.5ms
24:	learn: 0.3924658	total: 29.1ms	remaining: 87.2ms
25:	learn: 0.3897360	total: 30.2ms	remaining: 86ms
26:	learn: 0.3871082	total: 31.3ms	remaining: 84.7ms
27:	learn: 0.3850757	total: 32.4ms	remaining: 83.4ms
28:	learn: 0.3830391	total: 33.5ms	remaining: 82ms
29:	learn: 0.3814138	total: 34.6ms	remaining: 80.8ms
30:	learn: 0.3801030	total: 35.7ms	remaining: 79.4ms
31:	learn: 0.3786430	total: 36.9ms	remaining: 78.5ms
32:	learn: 0.3760069	total: 38.3ms	remaining: 77.7ms
33:	learn: 0.3750691	total: 39.7ms	remaining: 77.1ms
34:	learn: 0.3733958	total: 41.4ms	remaining: 76.8ms
35:

C:\Users\Golibjon\AppData\Local\Temp\ipykernel_18052\1622768231.py:23: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model1.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_18052\1622768231.py:23: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model1.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_18052\1622768231.py:23: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model1.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_18052\1622768231.py:23: FutureWarning: Series.ravel is d


Random Forest - Accuracy: 0.8258, F1-Score: 0.7896
Gradient Boosting - Accuracy: 0.8145, F1-Score: 0.7698
XGBoost - Accuracy: 0.8107, F1-Score: 0.7683
LightGBM - Accuracy: 0.8127, F1-Score: 0.7702
CatBoost - Accuracy: 0.8124, F1-Score: 0.7725
SVM - Accuracy: 0.7491, F1-Score: 0.7097
Dummy - Accuracy: 0.4833, F1-Score: 0.4565


C:\Users\Golibjon\AppData\Local\Temp\ipykernel_18052\1622768231.py:23: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model1.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_18052\1622768231.py:23: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model1.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_18052\1622768231.py:23: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model1.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_18052\1622768231.py:23: FutureWarning: Series.ravel is d

## Label: LONG_INTERACTION

In [40]:
print(encoded_data['LONG_INTERACTION_availability'].value_counts())

LONG_INTERACTION_availability
False    1471
True     1434
Name: count, dtype: int64


### Model building and LOSO CV

In [41]:
# Prepare features (X) and target variable (y) for predicting LONG_INTERACTION
X = encoded_data.drop(columns=['LONG_INTERACTION_availability', 'SHORT_INTERACTION_availability','pid'], axis=1)
y = encoded_data['LONG_INTERACTION_availability']
groups = encoded_data['pid']

# Initialize Leave-One-Group-Out cross-validator
logo = LeaveOneGroupOut()

# Initialize SMOTE for oversampling to handle class imbalance
smote = SMOTE(random_state=42)

# Define models to evaluate
models2 = {
    'Random Forest': RandomForestClassifier(n_estimators=100, max_depth=5, random_state=42),
    'Gradient Boosting': GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42),
    'XGBoost': XGBClassifier(n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42),
    'LightGBM': LGBMClassifier(n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42),
    'CatBoost': CatBoostClassifier(iterations=100, depth=5, learning_rate=0.1, loss_function='Logloss', cat_features=[0], random_seed=42),
    'SVM': SVC(random_state=42),
    'Dummy': DummyClassifier(strategy="stratified", random_state=42)
}

results2 = {}

# Loop over each model
for model_name, model2 in models2.items():
    accuracies = []
    f1_scores = []

    # Loop over each fold in Leave-One-Group-Out cross-validation
    for i, (train_index, test_index) in enumerate(logo.split(X, y, groups)):
        # Split the data into training and testing sets for the current fold
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        # Apply SMOTE to the training data to balance class distribution
        X_train_oversampled, y_train_oversampled = smote.fit_resample(X_train, y_train)

        # Train the model on the oversampled training data
        model2.fit(X_train_oversampled, y_train_oversampled.ravel())

        # Predict the target on the test data
        y_pred = model2.predict(X_test)

        # Evaluate the prediction
        accuracy = accuracy_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred, average='macro')

        accuracies.append(accuracy)
        f1_scores.append(f1)

    # Compute average accuracy and F1-score across all folds
    avg_accuracy = sum(accuracies) / len(accuracies)
    avg_f1_score = sum(f1_scores) / len(f1_scores)

    results2[model_name] = {
        'Average Accuracy': avg_accuracy,
        'Average F1-Score (macro)': avg_f1_score
    }

print()
for model_name, metrics in results2.items():
    print(f'{model_name} - Accuracy: {metrics["Average Accuracy"]:.4f}, F1-Score: {metrics["Average F1-Score (macro)"]:.4f}')

C:\Users\Golibjon\AppData\Local\Temp\ipykernel_18052\2714654808.py:40: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model2.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_18052\2714654808.py:40: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model2.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_18052\2714654808.py:40: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model2.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_18052\2714654808.py:40: FutureWarning: Series.ravel is d

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 1428, number of negative: 1428
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000589 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 89
[LightGBM] [Info] Number of data points in the train set: 2856, number of used features: 18
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with pos

C:\Users\Golibjon\AppData\Local\Temp\ipykernel_18052\2714654808.py:40: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model2.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_18052\2714654808.py:40: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model2.fit(X_train_oversampled, y_train_oversampled.ravel())


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 1447, number of negative: 1447
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000529 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 91
[LightGBM] [Info] Number of data points in the train set: 2894, number of used features: 19
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with pos

C:\Users\Golibjon\AppData\Local\Temp\ipykernel_18052\2714654808.py:40: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model2.fit(X_train_oversampled, y_train_oversampled.ravel())


...

C:\Users\Golibjon\AppData\Local\Temp\ipykernel_18052\2714654808.py:40: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model2.fit(X_train_oversampled, y_train_oversampled.ravel())


0:	learn: 0.6740596	total: 1.14ms	remaining: 112ms
1:	learn: 0.6607891	total: 2.4ms	remaining: 118ms
2:	learn: 0.6479686	total: 3.48ms	remaining: 113ms
3:	learn: 0.6364282	total: 4.48ms	remaining: 107ms
4:	learn: 0.6240159	total: 5.45ms	remaining: 104ms
5:	learn: 0.6141753	total: 6.39ms	remaining: 100ms
6:	learn: 0.6064643	total: 7.39ms	remaining: 98.1ms
7:	learn: 0.5985491	total: 8.58ms	remaining: 98.7ms
8:	learn: 0.5921782	total: 9.79ms	remaining: 99ms
9:	learn: 0.5870570	total: 11ms	remaining: 98.6ms
10:	learn: 0.5807881	total: 12.1ms	remaining: 98ms
11:	learn: 0.5755052	total: 13.3ms	remaining: 97.2ms
12:	learn: 0.5725163	total: 14.3ms	remaining: 95.9ms
13:	learn: 0.5686124	total: 15.5ms	remaining: 95.1ms
14:	learn: 0.5658730	total: 16.6ms	remaining: 94.1ms
15:	learn: 0.5638715	total: 17.8ms	remaining: 93.2ms
16:	learn: 0.5604566	total: 18.9ms	remaining: 92.5ms
17:	learn: 0.5571894	total: 20.1ms	remaining: 91.4ms
18:	learn: 0.5551139	total: 21.2ms	remaining: 90.5ms
19:	learn: 0.553

C:\Users\Golibjon\AppData\Local\Temp\ipykernel_18052\2714654808.py:40: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model2.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_18052\2714654808.py:40: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model2.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_18052\2714654808.py:40: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model2.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_18052\2714654808.py:40: FutureWarning: Series.ravel is d


Random Forest - Accuracy: 0.6878, F1-Score: 0.6335
Gradient Boosting - Accuracy: 0.6658, F1-Score: 0.5974
XGBoost - Accuracy: 0.6738, F1-Score: 0.6060
LightGBM - Accuracy: 0.6728, F1-Score: 0.6024
CatBoost - Accuracy: 0.6827, F1-Score: 0.6072
SVM - Accuracy: 0.6530, F1-Score: 0.6044
Dummy - Accuracy: 0.4883, F1-Score: 0.4562


C:\Users\Golibjon\AppData\Local\Temp\ipykernel_18052\2714654808.py:40: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model2.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_18052\2714654808.py:40: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model2.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_18052\2714654808.py:40: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model2.fit(X_train_oversampled, y_train_oversampled.ravel())


### K-Fold CV

In [42]:
from sklearn.model_selection import KFold

# Initialize 5-fold cross-validator
kfold = KFold(n_splits=5, shuffle=False)

results2 = {}

# Loop over each model
for model_name, model2 in models2.items():
    accuracies = []
    f1_scores = []

    # Loop over each fold in 5-fold cross-validation
    for train_index, test_index in kfold.split(X, y):
        # Split the data into training and testing sets for the current fold
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        # Apply SMOTE to the training data to balance class distribution
        X_train_oversampled, y_train_oversampled = smote.fit_resample(X_train, y_train)

        # Train the model on the oversampled training data
        model2.fit(X_train_oversampled, y_train_oversampled.ravel())

        # Predict the target on the test data
        y_pred = model2.predict(X_test)

        # Evaluate the prediction
        accuracy = accuracy_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred, average='macro')

        accuracies.append(accuracy)
        f1_scores.append(f1)

    # Compute average accuracy and F1-score across all folds
    avg_accuracy = sum(accuracies) / len(accuracies)
    avg_f1_score = sum(f1_scores) / len(f1_scores)

    results2[model_name] = {
        'Average Accuracy': avg_accuracy,
        'Average F1-Score (macro)': avg_f1_score
    }

print()
for model_name, metrics in results2.items():
    print(f'{model_name} - Accuracy: {metrics["Average Accuracy"]:.4f}, F1-Score: {metrics["Average F1-Score (macro)"]:.4f}')

C:\Users\Golibjon\AppData\Local\Temp\ipykernel_18052\715139799.py:23: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model2.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_18052\715139799.py:23: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model2.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_18052\715139799.py:23: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model2.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_18052\715139799.py:23: FutureWarning: Series.ravel is depre

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 1191, number of negative: 1191
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000521 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 86
[LightGBM] [Info] Number of data points in the train set: 2382, number of used features: 18
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with pos

C:\Users\Golibjon\AppData\Local\Temp\ipykernel_18052\715139799.py:23: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model2.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_18052\715139799.py:23: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model2.fit(X_train_oversampled, y_train_oversampled.ravel())


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

C:\Users\Golibjon\AppData\Local\Temp\ipykernel_18052\715139799.py:23: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model2.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_18052\715139799.py:23: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model2.fit(X_train_oversampled, y_train_oversampled.ravel())


...

C:\Users\Golibjon\AppData\Local\Temp\ipykernel_18052\715139799.py:23: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model2.fit(X_train_oversampled, y_train_oversampled.ravel())


0:	learn: 0.6745939	total: 1.31ms	remaining: 130ms
1:	learn: 0.6602890	total: 2.6ms	remaining: 127ms
2:	learn: 0.6471079	total: 4.25ms	remaining: 137ms
3:	learn: 0.6340331	total: 5.47ms	remaining: 131ms
4:	learn: 0.6237244	total: 6.58ms	remaining: 125ms
5:	learn: 0.6191003	total: 7.11ms	remaining: 111ms
6:	learn: 0.6113462	total: 8.2ms	remaining: 109ms
7:	learn: 0.6058677	total: 9.29ms	remaining: 107ms
8:	learn: 0.6001750	total: 10.4ms	remaining: 105ms
9:	learn: 0.5921429	total: 11.5ms	remaining: 103ms
10:	learn: 0.5865931	total: 12.5ms	remaining: 101ms
11:	learn: 0.5825531	total: 13.6ms	remaining: 99.7ms
12:	learn: 0.5789265	total: 14.7ms	remaining: 98.3ms
13:	learn: 0.5752227	total: 16.1ms	remaining: 99ms
14:	learn: 0.5706012	total: 17.5ms	remaining: 98.9ms
15:	learn: 0.5672086	total: 18.7ms	remaining: 98.4ms
16:	learn: 0.5639392	total: 19.9ms	remaining: 97.2ms
17:	learn: 0.5614034	total: 21.2ms	remaining: 96.5ms
18:	learn: 0.5592911	total: 22.6ms	remaining: 96.5ms
19:	learn: 0.55768

C:\Users\Golibjon\AppData\Local\Temp\ipykernel_18052\715139799.py:23: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model2.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_18052\715139799.py:23: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model2.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_18052\715139799.py:23: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model2.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_18052\715139799.py:23: FutureWarning: Series.ravel is depre


Random Forest - Accuracy: 0.6675, F1-Score: 0.6652
Gradient Boosting - Accuracy: 0.6664, F1-Score: 0.6624
XGBoost - Accuracy: 0.6657, F1-Score: 0.6610
LightGBM - Accuracy: 0.6713, F1-Score: 0.6662
CatBoost - Accuracy: 0.6699, F1-Score: 0.6655
SVM - Accuracy: 0.5928, F1-Score: 0.5921
Dummy - Accuracy: 0.4926, F1-Score: 0.4908


C:\Users\Golibjon\AppData\Local\Temp\ipykernel_18052\715139799.py:23: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model2.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_18052\715139799.py:23: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model2.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_18052\715139799.py:23: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model2.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_18052\715139799.py:23: FutureWarning: Series.ravel is depre